---
title: Assignment 03
author:
  - name: Taylor Luckenbill
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
number-sections: true
date: '2024-11-21'
format:
  html:
    theme: cerulean
    toc: true
    toc-depth: 2
date-modified: today
date-format: long
execute:
  echo: false
  eval: false
  freeze: auto
---

In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from pyspark.sql import SparkSession
import re
import numpy as np
import plotly.graph_objects as go
from pyspark.sql.functions import col, split, explode, regexp_replace, transform, when
from pyspark.sql import functions as F
from pyspark.sql.functions import col, monotonically_increasing_id

In [2]:
np.random.seed(2)

pio.renderers.default = "notebook"

# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("data/lightcast_job_postings.csv")
df.createOrReplaceTempView("job_postings")

# Show Schema and Sample Data
#print("---This is Diagnostic check, No need to print it in the final doc---")

#df.printSchema() # comment this line when rendering the submission
df.show(5)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/23 23:11:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/23 23:12:11 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-----------------+----------------------+----------+--------+---------+--------+--------------------+--------------------+--------------------+-----------+-------------------+--------------------+--------------------+---------------+----------------+--------+--------------------+-----------+-------------------+----------------+---------------------+-------------+-------------------+-------------+------------------+---------------+--------------------+--------------------+--------------------+-------------+------+-----------+----------------+-------------------+---------+-----------+--------------------+--------------------+-------------+------+--------------+-----+--------------------+-----+----------+---------------+--------------------+---------------+--------------------+------------+--------------------+------------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+------

In [15]:
filtered_df = df.filter(col("SALARY").isNotNull() 
                        & (col("SALARY") != 0) 
                        & col("MIN_YEARS_EXPERIENCE").isNotNull() )
filtered_df.show(30)

+--------------------+-----------------+----------------------+----------+---------+---------+--------+--------------------+--------------------+--------------------+-----------+-------------------+--------------------+--------------------+---------------+----------------+--------+--------------------+--------------------+-------------------+--------------------+---------------------+-------------+-------------------+-------------+------------------+---------------+--------------------+--------------------+--------------------+-------------+------+-----------+----------------+-------------------+---------+-----------+--------------------+--------------------+--------------------+------+--------------------+-----+--------------------+-----+--------------+---------------+--------------------+---------------+--------------------+------------+--------------------+------------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+---

In [24]:
grouped_df_pd = filtered_df.toPandas()

grouped_df_pd = (
    grouped_df_pd
    .groupby("MIN_YEARS_EXPERIENCE")
    .agg(
        median_salary=("SALARY", "median"),
        count=("ID", "count")
    )
    .reset_index()
)

grouped_df_pd.head(20)

,MIN_YEARS_EXPERIENCE,median_salary,count
0,0,75556.0,282
1,1,73126.0,1351
2,2,89596.0,3253
3,3,98800.0,2922
4,4,100000.0,1635
5,5,123800.0,5431
6,6,126000.0,1336
7,7,134149.0,1373
8,8,145450.0,1715
9,9,132860.0,212
